In [ ]:
! pip install einops

In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
import torch
import matplotlib.pyplot as plt
from qwen_vl_utils import process_vision_info

device = 'cuda'
model_path = 'Qwen/Qwen2.5-VL-3B-Instruct'

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
                model_path,
                torch_dtype=torch.bfloat16,
                attn_implementation="eager",
                device_map='auto',
            ).eval()
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True, padding_side='left', use_fast=True)

image_path = './images/demo1.png'
question = 'what is the date of the photo?'

messages_query = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": f"{question} Answer:"},
        ],
    }
]

image_inputs, _ = process_vision_info(messages_query)

text_query = processor.apply_chat_template(
    messages_query,
    tokenize=False,
    add_generation_prompt=True
)

inputs = processor(
    text=[text_query],
    images=image_inputs,
    padding=True,
    return_tensors="pt",
).to(device)

messages_general = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": "Write a general description of the image. Answer:"},
        ],
    }
]

text_general = processor.apply_chat_template(
    messages_general,
    tokenize=False,
    add_generation_prompt=True
)

general_inputs = processor(
    text=[text_general],
    images=image_inputs,
    padding=True,
    return_tensors="pt",
).to(device)

image_inputs_aux = processor.image_processor(images=image_inputs)
output_shape = image_inputs_aux["image_grid_thw"].numpy().squeeze(0)[1:]/2
output_shape = output_shape.astype(int)

with torch.no_grad():
    vision_start_token_id = processor.tokenizer.convert_tokens_to_ids('<|vision_start|>')
    vision_end_token_id = processor.tokenizer.convert_tokens_to_ids('<|vision_end|>')    
    pos = inputs['input_ids'].tolist()[0].index(vision_start_token_id) + 1
    pos_end = inputs['input_ids'].tolist()[0].index(vision_end_token_id)

    output = model(**inputs, output_attentions=True)
    general_output = model(**general_inputs, output_attentions=True)

    fig, axes = plt.subplots(6, 6, figsize=(10.8, 16))
    for i, ax in enumerate(axes.flatten()):
        att = output.attentions[i][0, :, -1, pos:pos_end].mean(dim=0)
        att = att.to(torch.float32).detach().cpu().numpy()

        general_att = general_output.attentions[i][0, :, -1, pos:pos_end].mean(dim=0)
        general_att = general_att.to(torch.float32).detach().cpu().numpy()

        att = att / general_att

        ax.imshow(att.reshape(output_shape), cmap='viridis', interpolation='nearest')
        ax.set_title(f'Layer {i+1}')
        ax.axis('off')

    plt.tight_layout()    
    plt.show()    